In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import LSTM
from tensorflow.keras.metrics import *
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.regularizers import l2

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[3], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [9]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [11]:
run_experiment(X,y)

Epoch 1/50
8/8 [==============================] - 1s 72ms/step - loss: 1.3595 - accuracy: 0.5071 - precision: 0.4714 - recall: 0.4229 - auc: 0.4981
Epoch 2/50
8/8 [==============================] - 1s 68ms/step - loss: 1.2844 - accuracy: 0.5170 - precision: 0.4835 - recall: 0.4439 - auc: 0.5248
Epoch 3/50
8/8 [==============================] - 1s 65ms/step - loss: 1.2228 - accuracy: 0.5323 - precision: 0.5013 - recall: 0.4673 - auc: 0.5570
Epoch 4/50
8/8 [==============================] - 1s 63ms/step - loss: 1.1555 - accuracy: 0.5926 - precision: 0.5711 - recall: 0.5257 - auc: 0.6337: 0s - loss: 1.1891 - accuracy: 0.5430 - precision: 0.5046 - recall: 0.4661 - auc: 
Epoch 5/50
8/8 [==============================] - 1s 68ms/step - loss: 1.0799 - accuracy: 0.6594 - precision: 0.6333 - recall: 0.6495 - auc: 0.7295
Epoch 6/50
8/8 [==============================] - 0s 59ms/step - loss: 0.9533 - accuracy: 0.7842 - precision: 0.7667 - recall: 0.7757 - auc: 0.8589
Epoch 7/50
8/8 [=============

8/8 [==============================] - 1s 80ms/step - loss: 1.1098 - accuracy: 0.6528 - precision_1: 0.6231 - recall_1: 0.6565 - auc_1: 0.7218
Epoch 5/50
8/8 [==============================] - 1s 64ms/step - loss: 0.9827 - accuracy: 0.7809 - precision_1: 0.7426 - recall_1: 0.8154 - auc_1: 0.8522
Epoch 6/50
8/8 [==============================] - 0s 56ms/step - loss: 0.8147 - accuracy: 0.8543 - precision_1: 0.8315 - recall_1: 0.8645 - auc_1: 0.9288
Epoch 7/50
8/8 [==============================] - 1s 64ms/step - loss: 0.6588 - accuracy: 0.9124 - precision_1: 0.8867 - recall_1: 0.9322 - auc_1: 0.9612
Epoch 8/50
8/8 [==============================] - 0s 56ms/step - loss: 0.5554 - accuracy: 0.9277 - precision_1: 0.9289 - recall_1: 0.9159 - auc_1: 0.9737
Epoch 9/50
8/8 [==============================] - 0s 53ms/step - loss: 0.4543 - accuracy: 0.9660 - precision_1: 0.9649 - recall_1: 0.9626 - auc_1: 0.9866
Epoch 10/50
8/8 [==============================] - 0s 60ms/step - loss: 0.3886 - accura

8/8 [==============================] - 1s 78ms/step - loss: 0.8328 - accuracy: 0.8719 - precision_2: 0.8642 - recall_2: 0.8621 - auc_2: 0.9327
Epoch 7/50
8/8 [==============================] - 1s 72ms/step - loss: 0.6789 - accuracy: 0.9157 - precision_2: 0.9016 - recall_2: 0.9206 - auc_2: 0.9642
Epoch 8/50
8/8 [==============================] - 1s 82ms/step - loss: 0.5478 - accuracy: 0.9398 - precision_2: 0.9451 - recall_2: 0.9252 - auc_2: 0.9792
Epoch 9/50
8/8 [==============================] - 1s 67ms/step - loss: 0.5155 - accuracy: 0.9332 - precision_2: 0.9297 - recall_2: 0.9276 - auc_2: 0.9762
Epoch 10/50
8/8 [==============================] - 1s 82ms/step - loss: 0.4690 - accuracy: 0.9365 - precision_2: 0.9343 - recall_2: 0.9299 - auc_2: 0.9807
Epoch 11/50
8/8 [==============================] - 1s 74ms/step - loss: 0.3661 - accuracy: 0.9748 - precision_2: 0.9634 - recall_2: 0.9836 - auc_2: 0.9940
Epoch 12/50
8/8 [==============================] - 1s 72ms/step - loss: 0.3219 - accu

8/8 [==============================] - 0s 62ms/step - loss: 0.5624 - accuracy: 0.9310 - precision_3: 0.9294 - recall_3: 0.9229 - auc_3: 0.9782
Epoch 9/50
8/8 [==============================] - 1s 75ms/step - loss: 0.4786 - accuracy: 0.9518 - precision_3: 0.9507 - recall_3: 0.9463 - auc_3: 0.9879
Epoch 10/50
8/8 [==============================] - 1s 72ms/step - loss: 0.4124 - accuracy: 0.9726 - precision_3: 0.9697 - recall_3: 0.9720 - auc_3: 0.9935
Epoch 11/50
8/8 [==============================] - 0s 50ms/step - loss: 0.3685 - accuracy: 0.9737 - precision_3: 0.9720 - recall_3: 0.9720 - auc_3: 0.9968
Epoch 12/50
8/8 [==============================] - 0s 62ms/step - loss: 0.3256 - accuracy: 0.9890 - precision_3: 0.9838 - recall_3: 0.9930 - auc_3: 0.9952
Epoch 13/50
8/8 [==============================] - 1s 67ms/step - loss: 0.2883 - accuracy: 0.9901 - precision_3: 0.9929 - recall_3: 0.9860 - auc_3: 0.9986
Epoch 14/50
8/8 [==============================] - 1s 67ms/step - loss: 0.2554 - ac

8/8 [==============================] - 1s 76ms/step - loss: 0.4574 - accuracy: 0.9562 - precision_4: 0.9663 - recall_4: 0.9393 - auc_4: 0.9867
Epoch 11/50
8/8 [==============================] - 1s 68ms/step - loss: 0.4256 - accuracy: 0.9496 - precision_4: 0.9463 - recall_4: 0.9463 - auc_4: 0.9893
Epoch 12/50
8/8 [==============================] - 0s 58ms/step - loss: 0.3689 - accuracy: 0.9682 - precision_4: 0.9586 - recall_4: 0.9743 - auc_4: 0.9955
Epoch 13/50
8/8 [==============================] - 1s 75ms/step - loss: 0.3093 - accuracy: 0.9836 - precision_4: 0.9814 - recall_4: 0.9836 - auc_4: 0.9989
Epoch 14/50
8/8 [==============================] - 0s 58ms/step - loss: 0.2799 - accuracy: 0.9858 - precision_4: 0.9837 - recall_4: 0.9860 - auc_4: 0.9988
Epoch 15/50
8/8 [==============================] - 0s 56ms/step - loss: 0.2481 - accuracy: 0.9912 - precision_4: 0.9907 - recall_4: 0.9907 - auc_4: 0.9995
Epoch 16/50
8/8 [==============================] - 1s 67ms/step - loss: 0.2220 - a

In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3),input_shape=(None,n_length,n_features)))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3)))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(50,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)
    
run_experiment_2(X,y)

Epoch 1/50
8/8 [==============================] - 0s 53ms/step - loss: 1.3567 - accuracy: 0.5246 - precision_5: 0.4922 - recall_5: 0.4416 - auc_5: 0.5483
Epoch 2/50
8/8 [==============================] - 1s 71ms/step - loss: 1.2328 - accuracy: 0.6407 - precision_5: 0.6096 - recall_5: 0.6495 - auc_5: 0.7021
Epoch 3/50
8/8 [==============================] - 1s 63ms/step - loss: 1.1109 - accuracy: 0.7426 - precision_5: 0.7169 - recall_5: 0.7453 - auc_5: 0.8104
Epoch 4/50
8/8 [==============================] - 1s 67ms/step - loss: 0.9273 - accuracy: 0.8291 - precision_5: 0.8192 - recall_5: 0.8154 - auc_5: 0.9121
Epoch 5/50
8/8 [==============================] - 1s 79ms/step - loss: 0.7845 - accuracy: 0.8905 - precision_5: 0.8744 - recall_5: 0.8949 - auc_5: 0.9488: 0s - loss: 0.8233 - accuracy: 0.8646 - precision_5: 0.7930 - recall_5: 0.9730 - auc_5: 
Epoch 6/50
8/8 [==============================] - 1s 65ms/step - loss: 0.6825 - accuracy: 0.9102 - precision_5: 0.8896 - recall_5: 0.9229 - a

Epoch 1/50
8/8 [==============================] - 1s 74ms/step - loss: 1.3884 - accuracy: 0.5115 - precision_6: 0.4729 - recall_6: 0.3668 - auc_6: 0.5134
Epoch 2/50
8/8 [==============================] - 1s 69ms/step - loss: 1.2910 - accuracy: 0.5542 - precision_6: 0.5190 - recall_6: 0.6706 - auc_6: 0.5864
Epoch 3/50
8/8 [==============================] - 1s 74ms/step - loss: 1.2152 - accuracy: 0.6068 - precision_6: 0.5844 - recall_6: 0.5584 - auc_6: 0.6500
Epoch 4/50
8/8 [==============================] - 1s 66ms/step - loss: 1.1518 - accuracy: 0.6342 - precision_6: 0.6526 - recall_6: 0.4696 - auc_6: 0.7125
Epoch 5/50
8/8 [==============================] - 1s 69ms/step - loss: 1.0219 - accuracy: 0.7536 - precision_6: 0.7411 - recall_6: 0.7290 - auc_6: 0.8311
Epoch 6/50
8/8 [==============================] - 1s 69ms/step - loss: 0.8265 - accuracy: 0.8795 - precision_6: 0.8768 - recall_6: 0.8645 - auc_6: 0.9441
Epoch 7/50
8/8 [==============================] - 1s 68ms/step - loss: 0.654

Epoch 1/50
8/8 [==============================] - 1s 66ms/step - loss: 1.3815 - accuracy: 0.4962 - precision_7: 0.4641 - recall_7: 0.4836 - auc_7: 0.5127
Epoch 2/50
8/8 [==============================] - 0s 61ms/step - loss: 1.2756 - accuracy: 0.5728 - precision_7: 0.5448 - recall_7: 0.5397 - auc_7: 0.6048
Epoch 3/50
8/8 [==============================] - 1s 71ms/step - loss: 1.1846 - accuracy: 0.6495 - precision_7: 0.6371 - recall_7: 0.5864 - auc_7: 0.7126
Epoch 4/50
8/8 [==============================] - 1s 85ms/step - loss: 1.0312 - accuracy: 0.7853 - precision_7: 0.7871 - recall_7: 0.7430 - auc_7: 0.8650
Epoch 5/50
8/8 [==============================] - 1s 75ms/step - loss: 0.8861 - accuracy: 0.8412 - precision_7: 0.8194 - recall_7: 0.8481 - auc_7: 0.9133
Epoch 6/50
8/8 [==============================] - 1s 72ms/step - loss: 0.7158 - accuracy: 0.9091 - precision_7: 0.8966 - recall_7: 0.9112 - auc_7: 0.9691
Epoch 7/50
8/8 [==============================] - 1s 67ms/step - loss: 0.580

Epoch 1/50
8/8 [==============================] - 0s 62ms/step - loss: 1.3966 - accuracy: 0.5257 - precision_8: 0.4938 - recall_8: 0.4650 - auc_8: 0.5197
Epoch 2/50
8/8 [==============================] - 0s 61ms/step - loss: 1.2829 - accuracy: 0.5816 - precision_8: 0.5453 - recall_8: 0.6472 - auc_8: 0.6121
Epoch 3/50
8/8 [==============================] - 1s 64ms/step - loss: 1.1703 - accuracy: 0.6648 - precision_8: 0.6473 - recall_8: 0.6262 - auc_8: 0.7372
Epoch 4/50
8/8 [==============================] - 1s 72ms/step - loss: 0.9889 - accuracy: 0.8171 - precision_8: 0.8000 - recall_8: 0.8131 - auc_8: 0.8987
Epoch 5/50
8/8 [==============================] - 1s 65ms/step - loss: 0.8975 - accuracy: 0.8467 - precision_8: 0.8172 - recall_8: 0.8668 - auc_8: 0.9117
Epoch 6/50
8/8 [==============================] - 0s 62ms/step - loss: 0.7145 - accuracy: 0.9189 - precision_8: 0.9041 - recall_8: 0.9252 - auc_8: 0.9744
Epoch 7/50
8/8 [==============================] - 1s 69ms/step - loss: 0.642

Epoch 1/50
8/8 [==============================] - 1s 74ms/step - loss: 1.3563 - accuracy: 0.5389 - precision_9: 0.5087 - recall_9: 0.4766 - auc_9: 0.5577
Epoch 2/50
8/8 [==============================] - 1s 73ms/step - loss: 1.2589 - accuracy: 0.5860 - precision_9: 0.5691 - recall_9: 0.4813 - auc_9: 0.6175
Epoch 3/50
8/8 [==============================] - 1s 63ms/step - loss: 1.0962 - accuracy: 0.7426 - precision_9: 0.7102 - recall_9: 0.7617 - auc_9: 0.8282
Epoch 4/50
8/8 [==============================] - 1s 72ms/step - loss: 0.9119 - accuracy: 0.8554 - precision_9: 0.8246 - recall_9: 0.8785 - auc_9: 0.9319
Epoch 5/50
8/8 [==============================] - 1s 70ms/step - loss: 0.7076 - accuracy: 0.9200 - precision_9: 0.9118 - recall_9: 0.9182 - auc_9: 0.9762
Epoch 6/50
8/8 [==============================] - 1s 74ms/step - loss: 0.5858 - accuracy: 0.9518 - precision_9: 0.9550 - recall_9: 0.9416 - auc_9: 0.9866
Epoch 7/50
8/8 [==============================] - 1s 83ms/step - loss: 0.501

 Accuracy: 79.039% (+/-12.193) 
 Precision: 71.511% (+/-25.935) 
 Recall: 58.409% (+/-36.375) 
 AUC: 85.795% (+/-16.624) 


In [13]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu'),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='selu'))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'selu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_3(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)
    
run_experiment_3(X,y)

Epoch 1/50
8/8 [==============================] - 1s 77ms/step - loss: 1.5360 - accuracy: 0.5279 - precision_10: 0.4962 - recall_10: 0.4556 - auc_10: 0.5329
Epoch 2/50
8/8 [==============================] - 1s 81ms/step - loss: 1.3687 - accuracy: 0.5696 - precision_10: 0.5331 - recall_10: 0.6589 - auc_10: 0.6199
Epoch 3/50
8/8 [==============================] - 1s 64ms/step - loss: 1.2239 - accuracy: 0.6681 - precision_10: 0.6667 - recall_10: 0.5841 - auc_10: 0.7354
Epoch 4/50
8/8 [==============================] - 1s 69ms/step - loss: 1.0853 - accuracy: 0.7525 - precision_10: 0.7512 - recall_10: 0.7056 - auc_10: 0.8448
Epoch 5/50
8/8 [==============================] - 1s 63ms/step - loss: 0.9414 - accuracy: 0.8467 - precision_10: 0.8333 - recall_10: 0.8411 - auc_10: 0.9176
Epoch 6/50
8/8 [==============================] - 1s 64ms/step - loss: 0.8393 - accuracy: 0.8806 - precision_10: 0.8684 - recall_10: 0.8785 - auc_10: 0.9509
Epoch 7/50
8/8 [==============================] - 0s 59ms/

Epoch 1/50
8/8 [==============================] - 0s 57ms/step - loss: 1.6146 - accuracy: 0.5367 - precision_11: 0.5057 - recall_11: 0.5140 - auc_11: 0.5549
Epoch 2/50
8/8 [==============================] - 0s 58ms/step - loss: 1.4022 - accuracy: 0.5619 - precision_11: 0.5293 - recall_11: 0.5911 - auc_11: 0.5886
Epoch 3/50
8/8 [==============================] - 0s 56ms/step - loss: 1.2498 - accuracy: 0.6648 - precision_11: 0.6597 - recall_11: 0.5888 - auc_11: 0.7134
Epoch 4/50
8/8 [==============================] - 1s 63ms/step - loss: 1.0784 - accuracy: 0.7634 - precision_11: 0.7573 - recall_11: 0.7290 - auc_11: 0.8413
Epoch 5/50
8/8 [==============================] - 0s 56ms/step - loss: 1.0091 - accuracy: 0.7941 - precision_11: 0.7703 - recall_11: 0.7991 - auc_11: 0.8756
Epoch 6/50
8/8 [==============================] - 1s 65ms/step - loss: 0.9057 - accuracy: 0.8543 - precision_11: 0.8471 - recall_11: 0.8411 - auc_11: 0.9210
Epoch 7/50
8/8 [==============================] - 1s 78ms/

Epoch 1/50
8/8 [==============================] - 0s 62ms/step - loss: 1.4075 - accuracy: 0.5816 - precision_12: 0.5575 - recall_12: 0.5210 - auc_12: 0.5947
Epoch 2/50
8/8 [==============================] - 0s 53ms/step - loss: 1.1895 - accuracy: 0.7141 - precision_12: 0.6811 - recall_12: 0.7336 - auc_12: 0.7819
Epoch 3/50
8/8 [==============================] - 0s 56ms/step - loss: 1.0544 - accuracy: 0.7831 - precision_12: 0.7614 - recall_12: 0.7827 - auc_12: 0.8641
Epoch 4/50
8/8 [==============================] - 0s 55ms/step - loss: 0.8969 - accuracy: 0.8565 - precision_12: 0.8414 - recall_12: 0.8551 - auc_12: 0.9308
Epoch 5/50
8/8 [==============================] - 0s 57ms/step - loss: 0.7616 - accuracy: 0.9036 - precision_12: 0.9048 - recall_12: 0.8879 - auc_12: 0.9681
Epoch 6/50
8/8 [==============================] - 0s 61ms/step - loss: 0.6605 - accuracy: 0.9376 - precision_12: 0.9206 - recall_12: 0.9486 - auc_12: 0.9877
Epoch 7/50
8/8 [==============================] - 0s 53ms/

Epoch 1/50
8/8 [==============================] - 0s 57ms/step - loss: 1.5361 - accuracy: 0.5674 - precision_13: 0.5363 - recall_13: 0.5701 - auc_13: 0.5674
Epoch 2/50
8/8 [==============================] - 0s 58ms/step - loss: 1.4125 - accuracy: 0.5411 - precision_13: 0.5106 - recall_13: 0.5047 - auc_13: 0.5709
Epoch 3/50
8/8 [==============================] - 0s 60ms/step - loss: 1.2788 - accuracy: 0.6331 - precision_13: 0.6126 - recall_13: 0.5911 - auc_13: 0.6707
Epoch 4/50
8/8 [==============================] - 0s 57ms/step - loss: 1.1005 - accuracy: 0.7459 - precision_13: 0.7333 - recall_13: 0.7196 - auc_13: 0.8166
Epoch 5/50
8/8 [==============================] - 0s 51ms/step - loss: 0.9671 - accuracy: 0.8050 - precision_13: 0.7765 - recall_13: 0.8201 - auc_13: 0.8912
Epoch 6/50
8/8 [==============================] - 0s 55ms/step - loss: 0.8296 - accuracy: 0.8806 - precision_13: 0.8552 - recall_13: 0.8972 - auc_13: 0.9452
Epoch 7/50
8/8 [==============================] - 0s 54ms/

Epoch 1/50
8/8 [==============================] - 0s 57ms/step - loss: 1.4606 - accuracy: 0.5509 - precision_14: 0.5211 - recall_14: 0.5187 - auc_14: 0.5780
Epoch 2/50
8/8 [==============================] - 0s 62ms/step - loss: 1.2014 - accuracy: 0.7065 - precision_14: 0.6702 - recall_14: 0.7360 - auc_14: 0.7727
Epoch 3/50
8/8 [==============================] - 0s 55ms/step - loss: 1.0638 - accuracy: 0.7777 - precision_14: 0.7368 - recall_14: 0.8178 - auc_14: 0.8629
Epoch 4/50
8/8 [==============================] - 0s 50ms/step - loss: 0.9386 - accuracy: 0.8258 - precision_14: 0.8150 - recall_14: 0.8131 - auc_14: 0.9155
Epoch 5/50
8/8 [==============================] - 0s 57ms/step - loss: 0.8100 - accuracy: 0.8959 - precision_14: 0.8955 - recall_14: 0.8808 - auc_14: 0.9578
Epoch 6/50
8/8 [==============================] - 0s 62ms/step - loss: 0.7319 - accuracy: 0.9080 - precision_14: 0.8981 - recall_14: 0.9065 - auc_14: 0.9727
Epoch 7/50
8/8 [==============================] - 0s 50ms/

 Accuracy: 59.651% (+/-3.020) 
 Precision: 43.786% (+/-8.607) 
 Recall: 10.909% (+/-3.262) 
 AUC: 61.162% (+/-5.177) 
